<a href="https://colab.research.google.com/github/du6293/DL_study/blob/main/3%EB%8B%A8%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 파이토치 토치비젼 설치
!pip3 install torch torchvision

## 텐서, 기울기, 장치(Tensor, Gradient, and Devices)

In [ ]:
import numpy as np
import torch

## 기울기 계산 기능을 끄고 기울기 계산을 시도하는 경우

In [ ]:
x = torch.tensor([1.,2.,3.])
y = torch.tensor([2.,3.,4.])

# x와 x의 기울기 계산 여부
print(x, x.requires_grad)

# 기울기 계산이 꺼져있으므로 backward를 하면 오류가 난다.
z = x + y
# z.sum().backward()

## 기울기 계산 기능을 키는 경우

In [ ]:
x = torch.tensor([1.,2.,3.],requires_grad = True)
y = torch.tensor([2.,3.,4.],requires_grad = True)
print(x,x.requires_grad)

# 잘 되는 것을 확인
z = x + y
z.sum().backward()

인퍼런스 모드

In [ ]:
print(x.requires_grad,y.requires_grad)

with torch.no_grad():
  z = x + y
  print(z.requires_grad)

## 텐서의 위치 할당

In [ ]:
# device 함수를 이용해 cpu 또는 몇 번 gpu에 올릴 것인지 지정할 수 있습니다.

cpu = torch.device('cpu')
gpu = torch.device('cuda')

# 텐서를 생성할 때, 장치를 지정할 수 있다.
x = torch.tensor([1.,2.,3.,], dtype = torch.float64, device = cpu, requires_grad = True)
print(x.device)

# to함수를 이용해 cpu에 만들어진 텐서를 gpu에 올릴 수 있습니다.
if torch.cuda.is_available():
  x = x.to(gpu)
  print(x.device)

In [ ]:
import torch

In [ ]:
x = torch.tensor(data=[2.0,3.0],requires_grad = True)
y = x **2
z = 2*y+3

In [ ]:
# https://pytorch.org/docs/stable/autograd.html?highlight=backward#torch.autograd.backward

# 목표값을 지정합니다. 
target = torch.tensor([3.0,4.0])

# z와 목표값의 절대값 차이를 계산합니다. 
# backward는 스칼라 값에 대해서 동작하기 때문에 길이 2짜리 텐서인 loss를 torch.sum을 통해 하나의 숫자로 바꿔줍니다.
loss = torch.sum(torch.abs(z-target))

# 그리고 스칼라 값이 된 loss에 대해 backward를 적용합니다.
loss.backward()

# 여기서 y와 z는 기울기가 None으로 나오는데 이는 x,y,z중에 x만이 leaf node이기 때문입니다.
print(x.grad, y.grad, z.grad)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt

In [ ]:
num_data = 1000
num_epoch = 500
noise = init.normal_(torch.FloatTensor(num_data,1),std = 0.2)

x = init.uniform_(torch.Tensor(num_data,1),-10,10)

y = 2*x+3

y_noise = y+noise

In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(x.numpy(), y_noise.numpy(), s=7, c="gray")
plt.axis([-12,12,-25,25])
plt.show()

## 모델, 손실함수, 최적화 함수 설정

In [ ]:
model = nn.Linear(1,1)
loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

## 모델 학습 및 중간 확인

In [ ]:
loss_arr = []
label = y_noise
for i in range(num_epoch):
  optimizer.zero_grad()
  output = model(x)
  loss = loss_func(output, label)
  loss.backward()
  optimizer.step()
  if i % 10 == 0:
    plt.scatter(x.detach().numpy(),output.detach().numpy())
    plt.axis([-10, 10 , -30, 30])
    plt.show()
    print(loss.data)
  loss_arr.append(loss.detach().numpy())

In [ ]:
plt.figure(figsize = (15,15))
plt.scatter(x.numpy(),y_noise.numpy(),s=5, c = "gray")
plt.scatter(x.detach().numpy(),output.detach().numpy(), s = 5, c = "red")
plt.axis([-10,10,-30,30])
plt.show()

## 손실 그래프

In [ ]:
# matplotlib의 plot함수를 이용해 손실이 어떻게 줄어가는지 확인
plt.plot(loss_arr)
plt.show()

## 학습 후 모델 변수 값 확인

In [ ]:
# 현재 모델은 weight와 bias를 변수로 가지고 있는데, 그 값들이 학습 후 실제 몇인지 수치적으로 확인.
param_list = list(model.parameters())
print("Weight:",param_list[0].item(),"\nBias:",param_list[1].item())